## Description

Exploring the TACO dataset (https://github.com/pedropro/TACO/tree/master) and transforming it into the format Spare-it uses (each entry in TACO has only one segmentation, and an image that has multiple pieces of trash would have multiple corresponding entries).

Examining the TACO Dataset:

In [45]:
import json
import requests

url = "https://raw.githubusercontent.com/pedropro/TACO/refs/heads/master/data/annotations.json"
response = requests.get(url)
data = response.json()

In [46]:
def print_json_keys(data, indent=0, seen=None):
    if seen is None:
        seen = set()  # Initialize a set to keep track of seen structures

    # Handle dictionaries
    if isinstance(data, dict):
        keys_tuple = tuple(data.keys())  # Use a tuple of keys to identify unique structures
        if keys_tuple in seen:
            return  # Skip if this structure has already been printed
        seen.add(keys_tuple)  # Add the structure to the set

        for key in data.keys():
            print('  ' * indent + str(key))
            print_json_keys(data[key], indent + 1, seen)

    # Handle lists
    elif isinstance(data, list):
        # Detect continuous ranges
        if len(data) > 1:
            print('  ' * indent + f"[{0} - {len(data) - 1}]")  # Print the range
        else:
            for index in range(len(data)):
                print('  ' * indent + f"[{index}]")  # Print individual index if only one

        for item in data:
            print_json_keys(item, indent + 1, seen)


In [47]:
print_json_keys(data)

info
  year
  version
  description
  contributor
  url
  date_created
images
  [0 - 1499]
    id
    width
    height
    file_name
    license
    flickr_url
    coco_url
    date_captured
    flickr_640_url
    id
    width
    height
    file_name
    license
    flickr_url
    flickr_640_url
    coco_url
    date_captured
annotations
  [0 - 4783]
    id
    image_id
    category_id
    segmentation
      [0]
        [0 - 157]
    area
    bbox
      [0 - 3]
    iscrowd
scene_annotations
  [0 - 4295]
    image_id
    background_ids
      [0]
licenses
categories
  [0 - 59]
    supercategory
    id
    name
scene_categories
  [0 - 6]
    id
    name


In [20]:
data["scene_annotations"][0]

{'image_id': 0, 'background_ids': [1]}

In [8]:
data["categories"][50]

{'supercategory': 'Pop tab', 'id': 50, 'name': 'Pop tab'}

In [24]:
data["scene_categories"][0]

{'id': 0, 'name': 'Clean'}

In [26]:
data["info"]

{'year': 2019,
 'version': None,
 'description': 'TACO',
 'contributor': None,
 'url': None,
 'date_created': '2019-12-19T16:11:15.258399+00:00'}

In [32]:
data["licenses"]

[]

In [37]:
data["annotations"][0]

{'id': 1,
 'image_id': 0,
 'category_id': 6,
 'segmentation': [[561.0,
   1238.0,
   568.0,
   1201.0,
   567.0,
   1175.0,
   549.0,
   1127.0,
   538.0,
   1089.0,
   519.0,
   1043.0,
   517.0,
   1005.0,
   523.0,
   964.0,
   529.0,
   945.0,
   520.0,
   896.0,
   525.0,
   862.0,
   536.0,
   821.0,
   554.0,
   769.0,
   577.0,
   727.0,
   595.0,
   678.0,
   596.0,
   585.0,
   588.0,
   346.0,
   581.0,
   328.0,
   569.0,
   306.0,
   570.0,
   276.0,
   576.0,
   224.0,
   560.0,
   205.0,
   564.0,
   170.0,
   578.0,
   154.0,
   608.0,
   136.0,
   649.0,
   127.0,
   688.0,
   127.0,
   726.0,
   129.0,
   759.0,
   141.0,
   784.0,
   153.0,
   792.0,
   177.0,
   788.0,
   193.0,
   782.0,
   209.0,
   792.0,
   238.0,
   802.0,
   271.0,
   802.0,
   294.0,
   791.0,
   319.0,
   789.0,
   360.0,
   794.0,
   395.0,
   810.0,
   529.0,
   819.0,
   609.0,
   841.0,
   675.0,
   882.0,
   728.0,
   916.0,
   781.0,
   928.0,
   802.0,
   938.0,
   834.0,
   940.0,
  

Transforming the TACO data and saving:

In [42]:
import json
import os
import datetime
from collections import defaultdict

def save_json(data, file_name, output_directory="transformed TACO annotations"):
    """
    Saves a JSON file to the specified output directory.
    """
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    file_path = os.path.join(output_directory, file_name)
    with open(file_path, 'w') as json_file:
        json.dump(data, json_file, indent=4)


In [43]:
def map_and_transform(taco_instance):
    """
    Transforms the entire TACO JSON into individual JSON files in spare-it format.
    Creates and saves a separate JSON file for each image.
    """
    # Current timestamp for date_created
    current_time = datetime.datetime.now().isoformat()

    # Step 1: Create a mapping from image_id to image information
    image_id_to_image = {image['id']: image for image in taco_instance['images']}

    # Step 2: Collect annotations for each image_id
    image_id_to_annotations = defaultdict(list)
    for annotation in taco_instance['annotations']:
        image_id = annotation['image_id']
        image_id_to_annotations[image_id].append(annotation)

    # Step 3: Process each image and its corresponding annotations
    for image_id, image_data in image_id_to_image.items():
        # Prepare the Structure A for this specific image
        spareit_JSON = {
            "info": {#same for all images
                "description": "Transformed TACO annotation",
                "url": "",
                "version": "",
                "year": 2019,
                "contributor": "",
                "date_created": current_time
            },
            "licenses": #same as spareit dataset
        [{
              "url": "",
              "id": 0,
              "name": ""
        }],
            "images": [],
            "type": "instances",
            "annotations": [],
            "categories": taco_instance.get("categories", []) #Todo: map categories
        }

        # Transform the image data for Structure A
        transformed_image = {
            'license': 0,
            'url': image_data.get('coco_url', ""),
            'file_name': image_data.get('file_name', "placeholder_file_name.jpg"),
            'height': image_data.get('height', 0),
            'width': image_data.get('width', 0),
            'date_captured': image_data.get('date_captured', ""),
            'id': image_id
        }
        spareit_JSON['images'].append(transformed_image)

        # Add the corresponding annotations for this image
        annotation_id = 0
        for annotation in image_id_to_annotations[image_id]:
            transformed_annotation = {
                'id': annotation_id,# same as spareit dataset
                'image_id': 0, # 0 for all, same as spareit dataset
                'category_id': annotation['category_id'], #Todo: map categories
                'segmentation': annotation['segmentation'],
                'area': annotation['area'],
                'bbox': annotation['bbox'],
                'iscrowd': annotation['iscrowd']
            }
            spareit_JSON['annotations'].append(transformed_annotation)
            annotation_id += 1
        # Step 4: Save the Structure A for this image as an individual JSON file
        image_file_name = f"image_{image_id}.json"
        save_json(spareit_JSON, image_file_name)


In [44]:
map_and_transform(data)